In [0]:
flightData2015 = spark.read.option("inferSchema","true")\
                  .option("header","true")\
                  .csv("/FileStore/tables/flight_dataset/2010_summary.csv")




In [0]:
flightData2015.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flightData2015.take(3)

Out[4]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [0]:
flightData2015.explain()

== Physical Plan ==
FileScan csv [DEST_COUNTRY_NAME#138,ORIGIN_COUNTRY_NAME#139,count#140] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/flight_dataset/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [0]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#140 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#140 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=153]
      +- FileScan csv [DEST_COUNTRY_NAME#138,ORIGIN_COUNTRY_NAME#139,count#140] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/flight_dataset/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [0]:
print("hello")

hello


In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "5")



### create sql table

In [0]:
flightData2015.createOrReplaceTempView("flight_data_2015")


In [0]:
sqlWay = spark.sql("""
SELECT ORIGIN_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY ORIGIN_COUNTRY_NAME
""")


In [0]:
sqlWay.show()

+--------------------+--------+
| ORIGIN_COUNTRY_NAME|count(1)|
+--------------------+--------+
|             Ireland|       1|
|           Singapore|       1|
|    Marshall Islands|       1|
|         Afghanistan|       1|
|              Angola|       1|
|            Portugal|       1|
|            Suriname|       1|
|          Cape Verde|       1|
|            Malaysia|       1|
|             Algeria|       1|
|              Panama|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
|             Liberia|       1|
|              France|       1|
|            Kiribati|       1|
|         New Zealand|       1|
|             Belgium|       1|
|             Austria|       1|
|             Nigeria|       1|
+--------------------+--------+
only showing top 20 rows



In [0]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#139], functions=[finalmerge_count(merge count#261L) AS count(1)#250L])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 0, Statistics(sizeInBytes=4.6 KiB, rowCount=131, isRuntime=true)
         +- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#139, 5), ENSURE_REQUIREMENTS, [plan_id=353]
            +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#139], functions=[partial_count(1) AS count#261L])
               +- FileScan csv [ORIGIN_COUNTRY_NAME#139] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/flight_dataset/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ORIGIN_COUNTRY_NAME:string>
+- == Initial Plan ==
   HashAggregate(keys=[ORIGIN_COUNTRY_NAME#139], functions=[finalmerge_count(merge count#261L) AS count(1)#250L])
   +- Exchange hashpartitioning(ORIGIN_COUNTRY_NAM

In [0]:
dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

In [0]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#138], functions=[finalmerge_count(merge count#277L) AS count(1)#272L])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 0, Statistics(sizeInBytes=4.4 KiB, rowCount=125, isRuntime=true)
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#138, 5), ENSURE_REQUIREMENTS, [plan_id=388]
            +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#138], functions=[partial_count(1) AS count#277L])
               +- FileScan csv [DEST_COUNTRY_NAME#138] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/flight_dataset/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
+- == Initial Plan ==
   HashAggregate(keys=[DEST_COUNTRY_NAME#138], functions=[finalmerge_count(merge count#277L) AS count(1)#272L])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#138, 5), ENS

In [0]:
from pyspark.sql.functions import max, desc


In [0]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           384932|
|           Canada|             8271|
|           Mexico|             6200|
|   United Kingdom|             1629|
|          Germany|             1392|
+-----------------+-----------------+



In [0]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           384932|
|           Canada|             8271|
|           Mexico|             6200|
|   United Kingdom|             1629|
|          Germany|             1392|
+-----------------+-----------------+



In [0]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#338L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#138,destination_total#338L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#138], functions=[finalmerge_sum(merge sum#342L) AS sum(count#140)#334L])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#138, 5), ENSURE_REQUIREMENTS, [plan_id=505]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#138], functions=[partial_sum(count#140) AS sum#342L])
            +- FileScan csv [DEST_COUNTRY_NAME#138,count#140] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/flight_dataset/2010_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


